In [100]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [101]:
N_EPOCHS = 70
N_TRIALS = 3
BATCH_SIZE = 256

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [117]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)
    
x, y = utils.load_data("../data/cifar-10-batches-py/data_batch_1")
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

train_size = x_train.shape[0]

# x_train = x_train / 255
# x_val = x_val / 255
# x_test = x_test / 255

In [118]:
noise = 30
x_train = np.clip(x_train + np.random.normal(0, noise, x_train.shape), 0, 255) / 255
x_val = np.clip(x_val + np.random.normal(0, noise, x_val.shape), 0, 255) / 255
x_test = np.clip(x_test + np.random.normal(0, noise, x_test.shape), 0, 255) / 255

In [119]:
assessment_model = ClassifierModel(output_shape=10, **models_hyperparameters["assessment_model"])

assessment_model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
        
assessment_model.fit(x_train, y_train, epochs=60, batch_size=BATCH_SIZE)

Epoch 1/60
28/28 [==============================] - 1s 28ms/step - loss: 2.5289 - accuracy: 0.1367
Epoch 2/60
28/28 [==============================] - 1s 28ms/step - loss: 2.2120 - accuracy: 0.1917
Epoch 3/60
28/28 [==============================] - 1s 27ms/step - loss: 2.0844 - accuracy: 0.2337
Epoch 4/60
28/28 [==============================] - 1s 28ms/step - loss: 2.0062 - accuracy: 0.2589
Epoch 5/60
28/28 [==============================] - 1s 29ms/step - loss: 1.9516 - accuracy: 0.2737
Epoch 6/60
28/28 [==============================] - 1s 29ms/step - loss: 1.8875 - accuracy: 0.3016
Epoch 7/60
28/28 [==============================] - 1s 30ms/step - loss: 1.8631 - accuracy: 0.3151
Epoch 8/60
28/28 [==============================] - 1s 28ms/step - loss: 1.8199 - accuracy: 0.3289
Epoch 9/60
28/28 [==============================] - 1s 34ms/step - loss: 1.7881 - accuracy: 0.3293
Epoch 10/60
28/28 [==============================] - 1s 29ms/step - loss: 1.7449 - accuracy: 0.3566
Epoch 11/

In [120]:
y_pred_assessment = assessment_model.predict(x_train, verbose=0)

losses_assessment = - np.array(utils.calculate_loss_per_sample(y_train, y_pred_assessment, loss))
losses_proba = np.exp(losses_assessment) / sum(np.exp(losses_assessment))

In [121]:
results = {}

# test_models = ["test_model_1", "test_model_2", "test_model_3"]
test_models = ["test_model_1"]

In [122]:
for test_model in test_models:
    print(test_model)
    model_scores = []

    for _ in tqdm(range(N_TRIALS)):
        model = ClassifierModel(output_shape=10, **models_hyperparameters[test_model])
        
        model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

        for i in range(N_EPOCHS):
            n_samples = int(np.tanh(4 * (i + 1) / N_EPOCHS) * train_size)
            samples_ids = np.random.choice(range(train_size), p=losses_proba, size=n_samples, replace=False)
            
            model.fit(
                x_train[samples_ids], y_train[samples_ids], 
                validation_data=(x_val, y_val),
                epochs=1, batch_size=BATCH_SIZE, verbose=1
            )
        
        _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=1)
        
        model_scores.append(accuracy)
    
    results[test_model] = model_scores

test_model_1


  0%|          | 0/3 [00:00<?, ?it/s]

6/6 [==============================] - 0s 22ms/step - loss: 1.6238 - accuracy: 0.4653


 33%|███▎      | 1/3 [03:19<06:38, 199.37s/it]

6/6 [==============================] - 0s 17ms/step - loss: 2.2275 - accuracy: 0.3820


 67%|██████▋   | 2/3 [06:31<03:15, 195.36s/it]

6/6 [==============================] - 0s 14ms/step - loss: 1.8400 - accuracy: 0.4453


100%|██████████| 3/3 [09:12<00:00, 184.30s/it]


In [126]:
results

{'test_model_1': [0.46533334255218506, 0.38199999928474426, 0.445333331823349]}

In [127]:
sum(results["test_model_1"]) / 3

0.4308888912200928